<a href="https://colab.research.google.com/github/ayuadmy/simpasi/blob/main/SIMPASI_MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Library Needed


In [3]:
!python --version

Python 3.7.13


In [4]:
pip install -q tensorflow-recommenders

     |████████████████████████████████| 85 kB 4.1 MB/s 
     |████████████████████████████████| 462 kB 45.4 MB/s 


In [5]:
import pathlib
import os
import pprint
import tempfile

from typing import Dict, Text

import csv
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

# MPASI Menu, Recipe, Ingredients Dataset

In [6]:
#Download datasets from google drive
#Bahan_Makanan
!gdown --id 1eenpLlfkok_acgmuIC2XI4tZCnMIkUrS\

#Move files to tmp (To be deleted after closing this google colab in some time)
!mv "/content/bahan_makanan.csv" "/tmp"

#Create directory of datasets
bahan_makanan_dir = "/tmp/bahan_makanan.csv"

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1eenpLlfkok_acgmuIC2XI4tZCnMIkUrS
To: /content/bahan_makanan.csv
100% 5.64k/5.64k [00:00<00:00, 6.10MB/s]


In [7]:
#Resep
!gdown --id 1miF99iHy0E0OUfkqP6dZVUccAmW_KpVT\

!mv "/content/resep.csv" "/tmp"
resep_dir = "/tmp/resep.csv"

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1miF99iHy0E0OUfkqP6dZVUccAmW_KpVT
To: /content/resep.csv
100% 3.57k/3.57k [00:00<00:00, 4.22MB/s]


In [8]:
#Komposisi
!gdown --id 1xufQ7DEIiKIcWWd0sOYXMLkjJQ8kYgvJ\

!mv "/content/komposisi_baru.csv" "/tmp"
komposisi_dir = "/tmp/komposisi_baru.csv"

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1xufQ7DEIiKIcWWd0sOYXMLkjJQ8kYgvJ
To: /content/komposisi_baru.csv
100% 9.23k/9.23k [00:00<00:00, 15.1MB/s]


In [9]:
#Menu Makanan
!gdown --id 1Elz56RH18uKtrV3DDePpT9SKOzhFGVPb\

!mv "/content/kombinasi_menu.csv" "/tmp"
kombinasi_menu_dir = "/tmp/kombinasi_menu.csv"

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1Elz56RH18uKtrV3DDePpT9SKOzhFGVPb
To: /content/kombinasi_menu.csv
100% 3.00M/3.00M [00:00<00:00, 207MB/s]


In [10]:
#Feedback_bahan
!gdown --id 12g229KO46uBZ8mSPRaSuAsxVjosmy7Ty\

!mv "/content/feedback_bahan.csv" "/tmp"
feedback_bahan_dir = "/tmp/feedback_bahan.csv"

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=12g229KO46uBZ8mSPRaSuAsxVjosmy7Ty
To: /content/feedback_bahan.csv
100% 1.37k/1.37k [00:00<00:00, 2.54MB/s]


In [11]:
feedback_bahan = pd.read_csv(feedback_bahan_dir)
feedback_bahan = feedback_bahan.iloc[:,[2,5]]
feedback_bahan.head()

,user_id,nama_bahan
0,1,Strawbery
1,1,Brokoli
2,1,Beras Merah
3,1,Kaldu Ayam
4,1,Tempe


In [12]:
nama_bahan = bahan_makanan.pop("nama_bahan")
nama_bahan.head()

NameError: ignored

In [13]:
user_id_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_id_vocabulary.adapt(feedback_bahan.loc[:,"user_id"].map(str))


nama_bahan_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
nama_bahan_vocabulary.adapt(nama_bahan)

NameError: ignored

In [14]:
class SIMPASIModel(tfrs.Model):
  # We derive from a custom base class to help reduce boilerplate. Under the hood,
  # these are still plain Keras Models.

  def __init__(
      self,
      user_model: tf.keras.Model,
      nama_bahan_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()

    # Set up user and movie representations.
    self.user_model = user_model
    self.nama_bahan_model = nama_bahan_model

    # Set up a retrieval task.
    self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.

    user_embeddings = self.user_model(features["user_id"])
    nama_bahan_embeddings = self.movie_model(features["nama_bahan"])

    return self.task(user_embeddings, nama_bahan_embeddings)

In [15]:
# Define user and movie models.
user_model = tf.keras.Sequential([
    user_id_vocabulary,
    tf.keras.layers.Embedding(user_id_vocabulary.vocab_size(), 64)
])
nama_bahan_model = tf.keras.Sequential([
    nama_bahan_vocabulary,
    tf.keras.layers.Embedding(nama_bahan_vocabulary.vocab_size(), 64)
])

# Define your objectives.
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    nama_bahan
  )
)

NameError: ignored

In [16]:
# Create a retrieval model.
model = SIMPASIModel(user_model, nama_bahan_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Train for 3 epochs.
model.fit(feedback_bahan, epochs=3)

# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    nama_bahan.batch(100).map(lambda title: (title, model.movie_model(title))))

# Get some recommendations.
_, titles = index(np.array(["42"]))
print(f"Top 3 recommendations for user 42: {titles[0, :3]}")

NameError: ignored

In [17]:
import pandas as pd

bahan_makanan = pd.read_csv(bahan_makanan_dir)
bahan_makanan.head()

,id_bahan,nama_bahan
0,0,Agar-Agar
1,1,Agar Laut
2,2,Akar Tonjong
3,3,Alpukat
4,4,Ampas Tahu


In [18]:
kombinasi_menu = pd.read_csv(kombinasi_menu_dir)

kombinasi_menu["malam"].fillna("", inplace = True) 
kombinasi_menu["snack1"].fillna("", inplace = True) 
kombinasi_menu["snack2"].fillna("", inplace = True) 

kombinasi_menu.head()

,id_kombinasi,pagi,siang,malam,snack1,snack2,bulan_6,bulan_7_8,bulan_9_11,bulan_12_24,total_energi,total_carbo,total_protein,total_lipid
0,1,Apel Rebus,Apel Rebus,,,,1,0,0,0,144.00,38.12,0.72,0.46
1,2,Bubur Apel,Bubur Apel,,,,1,0,0,0,144.50,34.52,0.66,0.42
2,3,Bubur Kacang Polong,Bubur Kacang Polong,,,,1,0,0,0,127.44,15.60,3.84,5.52
3,4,Bubur Kembang Kol,Bubur Kembang Kol,,,,1,0,0,0,143.20,21.12,9.46,2.32
4,5,Bubur Plum,Bubur Plum,,,,1,0,0,0,128.10,30.88,1.10,0.02


In [19]:
 import csv

dict_komposisi = {}

with open(komposisi_dir, mode="r") as data_komposisi:
  reader = csv.reader(data_komposisi)
  keys = next(reader)
  for row in reader:
    if row[0] not in dict_komposisi.keys():
      dict_komposisi[row[0]] = row[2]
    else:
      dict_komposisi[row[0]] = dict_komposisi[row[0]] + ", " + row[2]
    
data = {keys[0]: dict_komposisi.keys(), keys[2]: dict_komposisi.values()}
komposisi = pd.DataFrame.from_dict(data)
komposisi.head()

,id_resep,nama_bahan
0,0,"Blewah, Melon, Agar-Agar"
1,1,Apel
2,2,"Apel, Margarine, Oatmeal"
3,3,"Tahu Sutra, Pisang, Apel"
4,4,"Hati Ayam, Wortel, Kaldu Ayam, Beras, Ayam, Da..."


In [20]:
resep = pd.read_csv(resep_dir)
resep.head()

,id_resep,nama_resep
0,0,Agar-agar Isi Blewah
1,1,Apel Rebus
2,2,Apple Crumble
3,3,Apricot Tahu
4,4,Ayam Zukini Hati


In [28]:
#komposisi_resep = komposisi.set_index('id_resep').join(resep.set_index('id_resep'))
#komposisi_resep = pd.concat([komposisi, resep], ignore_index=True)
#komposisi_resep = pd.merge(komposisi, resep, how='inner', left = 'id_resep', right = 'id_resep')
komposisi['id_resep']=komposisi['id_resep'].astype(int)
komposisi_resep = komposisi.merge(resep, on=['id_resep', 'id_resep'], how='left')
#komposisi_resep

del(komposisi_resep['id_resep'])
komposisi_resep = komposisi_resep[['nama_resep', 'nama_bahan']]
komposisi_resep


,nama_resep,nama_bahan
0,Agar-agar Isi Blewah,"Blewah, Melon, Agar-Agar"
1,Apel Rebus,Apel
2,Apple Crumble,"Apel, Margarine, Oatmeal"
3,Apricot Tahu,"Tahu Sutra, Pisang, Apel"
4,Ayam Zukini Hati,"Hati Ayam, Wortel, Kaldu Ayam, Beras, Ayam, Da..."
...,...,...
155,Tim Saring,"Beras, Kaldu Ayam, Ikan Salmon, Beras Merah, W..."
156,Tumis Ikan Dori,"Bawang Merah, Wortel, Ketimun, Bawang Putih, T..."
157,Tuna Mornay,"Saus Pasta, Ikan Tuna, Roti Tawar, Keju, Krim ..."
158,Yoghurt Buah Pop,"Strawbery, Yoghurt"


# Simulasi Aplikasi SIMPASI - User Baru

Pada user baru, customer akan melewati tahap-tahap berikut akan berupa:

1.   Log-in/Sign-up dengan email dan passsword
2.   Meminta data bayi berupa nama, tanggal lahir, bahan yang pernah dicoba sebelumnya (suka/tidak suka/alergi)
3.   Mendapatkan rekomendasi makanan dari riwayat makanan yang disukai/tidak disukai/alergi




Sehingga, sistem rekomendasi harus melakukan:


1.   Meminta username/email/password untuk masuk ke akun
2.   Meminta nama, tanggal lahir, bahan yang pernah dicoba sebelumnya *(suka/tidak suka/alergi)
3.   Menghitung umur anak
4.   Menghitung EER anak
5.   Memfilter bahan berdasarkan alergi
6.   Memfilter resep berdasarkan umur dan EER
7.   Menampilkan kombinasi menu dari resep yang cocok




## I. *Contoh Profil User*

In [29]:
#Contoh
username = "brianjang"
password = "d8d807375d408c888d0ff9974a82edf4971ed91c0810ec4daa5e47b91bf0de76"
email = "ujang@mail.com"
nama_user = "Ujang Ramabrian"
nama_bayi = "Biyan"
tanggal_lahir = "2021/07/03"
berat_badan = 6.9

## II. Cek 8 Bahan (Suka, Tidak Suka, Alergi, Tidak tahu)


In [30]:
#Menampilkan Nama Bahan Berdasarkan ID
def ingredient_name(id):
  return bahan_makanan.loc[id, 'nama_bahan']

#driver code
print(ingredient_name(1))

Agar Laut


In [31]:
#Bahan Alergi Umum: Susu, Telur Ayam, Telur Bebek, Kacang Tanah, Kacang Kedelai, Udang, Kerang, Cumi-Cumi, Tepung Beras, Roti, Daging Sapi, Pisang, Alpukat
tes_bahan = ["Susu", "Telur Ayam", "Telur Bebek", "Kacang Tanah", "Kacang Kedelai", "Udang", "Kerang", "Cumi-Cumi", "Tepung Beras", "Roti Tawar", "Daging Sapi", "Pisang", "Alpukat"]
jumlah_tes = 8
alergi = []
tidak_suka = []
suka = []
tidak_tahu = []

import random
tes_bahan_random = random.sample(tes_bahan, jumlah_tes)

#Meminta 8 Feedback Bahan dari User
print("Pilih Suka/Tidak Suka/Alergi/Tidak Tahu Pada Bahan Yang Ditampilkan")
print("Ketik 1 = Suka, 2 = Tidak Suka, 3 = Alergi, 4 = Tidak Tahu")
for i in range(len(tes_bahan_random)):

  x = int(input(tes_bahan_random[i] + ": "))

  if x == 2:
    tidak_suka.append(tes_bahan_random[i])
  elif x == 3:
    alergi.append(tes_bahan_random[i])
  elif x == 4:
    tidak_tahu.append(tes_bahan_random[i])
  elif x == 1:
    suka.append(tes_bahan_random[i])

print(tidak_suka)
print(suka)
print(tidak_tahu)
print(alergi)

Pilih Suka/Tidak Suka/Alergi/Tidak Tahu Pada Bahan Yang Ditampilkan
Ketik 1 = Suka, 2 = Tidak Suka, 3 = Alergi, 4 = Tidak Tahu
Kerang: 3
Tepung Beras: 1
Telur Bebek: 2
Daging Sapi: 1
Kacang Kedelai: 4
Udang: 2
Telur Ayam: 3
Cumi-Cumi: 2
['Telur Bebek', 'Udang', 'Cumi-Cumi']
['Tepung Beras', 'Daging Sapi']
['Kacang Kedelai']
['Kerang', 'Telur Ayam']


## III. *Hitung Umur dan EER*

In [32]:
from datetime import datetime
from datetime import date

#Fungsi Menghitung Umur User
def ageCalculator(birthdate):
  my_birthdate = datetime.strptime(birthdate, "%Y/%m/%d")
  my_year = my_birthdate.year
  my_month = my_birthdate.month
  my_day = my_birthdate.year
  
  born = my_birthdate.date()
  today = date.today()
  
  years = ((today-born).total_seconds()/ (365.242*24*3600))
  yearsInt = int(years)
  months = (years-yearsInt)*12
  monthsInt = int(months)
  days = (months-monthsInt)*(365.242/12)
  daysInt = int(days)

  return yearsInt, monthsInt, daysInt
  

#Fungsi Menghitung EER (Estimated Energy Requirements)
def eerCalculator(year, month, day, weight):
  if month == 6:
    EER = (89*weight - 100) + 56
  elif 7 <= month <= 12:
    EER = (89*weight - 100) + 22
  elif 13 <= month <= 35:
    EER = (89*weight - 100) + 20
  
  return EER
  
#Driver Code
#Cari Umur User (Tahun, Bulan, Tanggal)
tahun, bulan, hari = ageCalculator(tanggal_lahir)
print("Sekarang kamu berumur {0} tahun, {1} bulan, {2} hari.".format(tahun , bulan, hari))

#Cari EER User (+-10%)
eer = eerCalculator(tahun, bulan, hari, berat_badan)
print("Kebutuhan energi harianmu: {0} - {1} Joule".format(int(eer*0.9), int(eer*1.1)))

Sekarang kamu berumur 0 tahun, 10 bulan, 26 hari.
Kebutuhan energi harianmu: 482 - 589 Joule


## IV. *Filtering Resep Berdasarkan Alergi*

In [35]:
#pattern = '|'.join(alergi)
pattern = '|'.join([f'(?i){bahan_alergi}' for bahan_alergi in suka])
#frame.a.str.contains(pattern)
#for i in alergi:
  #mask = (komposisi['nama_bahan'] == 'Agar-Agar')
bahan = ["Apel"]
#mask = komposisi['nama_bahan'].isin(bahan)
  #mask = menu_filter.loc[menu_filter['nama_bahan'] != i]
mask = komposisi_resep.nama_bahan.str.contains(pattern)
komposisi_resep[~mask]

,nama_resep,nama_bahan
0,Agar-agar Isi Blewah,"Blewah, Melon, Agar-Agar"
1,Apel Rebus,Apel
2,Apple Crumble,"Apel, Margarine, Oatmeal"
3,Apricot Tahu,"Tahu Sutra, Pisang, Apel"
4,Ayam Zukini Hati,"Hati Ayam, Wortel, Kaldu Ayam, Beras, Ayam, Da..."
...,...,...
155,Tim Saring,"Beras, Kaldu Ayam, Ikan Salmon, Beras Merah, W..."
156,Tumis Ikan Dori,"Bawang Merah, Wortel, Ketimun, Bawang Putih, T..."
157,Tuna Mornay,"Saus Pasta, Ikan Tuna, Roti Tawar, Keju, Krim ..."
158,Yoghurt Buah Pop,"Strawbery, Yoghurt"


## V. *Filtering Resep Berdasarkan umur dan EER*

## VI. *Model Sistem Rekomendasi Menu MPASI*